## 1. Parsing the System of 

### a) First lets open the file and read the lines

In [1]:
file = open('equasions.txt', 'r') 
equasionLines = file.readlines()
print(equasionLines)

['2x + 3y - z = 5\n', 'x - y + 4z = 6\n', '3x + y + 2z = 7\n']


### b) Now lets tranform the lines into a matrix by:
- Extraction each of the terms in their own string
- Replacing the term (x/y/z) with an empty string so that evaluating Number is not throwing an error
- Multiplying by -1 if the sign previous symbol is a minus
- $Assumption$: the first term has no space between the number and the minus (-2x not - 2x)

In [2]:
xTerm = 0
yTerm = 0
zTerm = 0
matrixA = []
matrixB = []
for equasionLine in equasionLines:
    lineElements = equasionLine.split()
    # normalizing the terms so that we can extract the number with a one line command
    if lineElements[0] == 'x':
        lineElements[0] = '1x'
    if lineElements[2] == 'y':
        lineElements[2] = '1y'
    if lineElements[4] == 'z':
        lineElements[4] = '1z'
    xTerm = int(lineElements[0].replace('x', ''))
    yTerm = int(lineElements[2].replace('y', ''))
    zTerm = int(lineElements[4].replace('z', ''))

    if lineElements[1] == '-':
        yTerm *= -1
    
    if lineElements[3] == '-':
        zTerm *= -1
        
    equalsValue = int(lineElements[6])
    matrixA.append([xTerm, yTerm, zTerm])
    matrixB.append(equalsValue)
print(matrixA, matrixB)

[[2, 3, -1], [1, -1, 4], [3, 1, 2]] [5, 6, 7]


## 2. Matrix and Vector Operations

### a) Function for calculating the Determinant of a matrix

In [3]:
def determinant (matrix): 
    if len(matrix) == 2:
        return matrix[0][0] * matrix[1][1] - matrix[0][1] * matrix[1][0]
        
    return \
    matrix[0][0] * (matrix[1][1] * matrix[2][2] - matrix[1][2] * matrix[2][1]) - \
    matrix[0][1] * (matrix[1][0] * matrix[2][2] - matrix[1][2] * matrix[2][0]) + \
    matrix[0][2] * (matrix[1][0] * matrix[2][1] - matrix[1][1] * matrix[2][0])

### b) Function for calculating Trace of a matrix 

In [4]:
def trace (matrix):
    return matrix[0][0] * matrix[1][1] * matrix[2][2]

### c) Function for calculating Euclidian norm of a vector

In [5]:
import math
def vectorNorm (vector):
    return math.sqrt(vector[0] * vector[0] + vector[1] * vector[1] + vector[2] * vector[2])

### d) Function for calculating the Transpose of a matrix (simple and plain implementation)

In [6]:
def transposeMatrix (matrix):
    newMatrix = [[matrix[0][0], matrix[1][0], matrix[2][0]], [matrix[0][1], matrix[1][1], matrix[2][1]], [matrix[0][2], matrix[1][2], matrix[2][2]]]
    return newMatrix

### e) Function for calculating the product of a Matrix and a Vector -> returns vector

In [7]:
def matrixVectorMultiplication (matrix, vector):
    resultVector = []
    for lineIndex in range(0, 3):
        sum = 0
        for colIndex in range(0, 3):
            sum += matrix[lineIndex][colIndex] * vector[colIndex]
        resultVector.append(sum)
    return resultVector
    
print(matrixVectorMultiplication(matrixA, matrixB))

[21, 27, 35]


### f) Additional functions for solving the system of equasions
- Replace column (needed for calculating the determinant of $A_x$, $A_y$, and $A_z$ for Cramer Rule

In [8]:
import copy
def replaceColumn(matrix, newColumn, columnIndexToReplace):
    matrixCopy = copy.deepcopy(matrix)
    for line in range (0, len(matrix)): 
        matrixCopy[line][columnIndexToReplace] = newColumn[line]
    return matrixCopy

## 3. Solving using Cramer's Rule 
### Preparation step 
- calculate the determinant of the matrix

In [9]:
matrixADetermiant = determinant(matrixA)

### a) Calculating x
- calculate $A_x$ matrix
- calculate x using the formula $x = \frac{\det(A_x)}{\det(A)}
$

In [10]:
matrixAx = replaceColumn(matrixA, matrixB, 0)
matrixAxDeterminant = determinant(matrixAx)
x_solution = matrixAxDeterminant / matrixADetermiant
print('x =', matrixAxDeterminant, '/', matrixADetermiant, '(float value):', x_solution)

x = 5 / 14 (float value): 0.35714285714285715


### b) Calculating y
- calculate $A_y$ matrix
- calculate y using the formula $y = \frac{\det(A_y)}{\det(A)}
$

In [11]:
matrixAy = replaceColumn(matrixA, matrixB, 1)
matrixAyDeterminant = determinant(matrixAy)
y_solution = matrixAyDeterminant / matrixADetermiant
print('y =', matrixAyDeterminant, '/', matrixADetermiant, '(float value):', y_solution)

y = 29 / 14 (float value): 2.0714285714285716


### c) Calculating z
- calculate $A_z$ matrix
- calculate z using the formula $z = \frac{\det(A_z)}{\det(A)}
$

In [12]:
matrixAz = replaceColumn(matrixA, matrixB, 2)
matrixAzDeterminant = determinant(matrixAz)
z_solution = matrixAzDeterminant / matrixADetermiant
print('z =', matrixAzDeterminant, '/', matrixADetermiant, '(float value):', z_solution)

z = 27 / 14 (float value): 1.9285714285714286


## 4. Solving using Inversion
- First we will create 2 new functions, one for computing the adjugate matrix and a helper function for that to calculate the minors
### a) Function for calculating a minor

In [13]:
def minor (initialMatrix, i, j):
    newMatrix = []
    for line in range(0, len(initialMatrix)):
        if line == i:
            continue
        newMatrix.append([])
        for column in range(0, len(initialMatrix)):
            if column == j:
                continue
            newMatrix[len(newMatrix) - 1].append(initialMatrix[line][column])
    return newMatrix

### b) Function for calculating the adjugate matrix

In [14]:
def adjugate (matrix):
    newMatrix = copy.deepcopy(matrix)
    for line in range(0, len(newMatrix)):
        for column in range(0, len(newMatrix)):
            minorMatrix = minor(matrix, line, column)
            minorDeterminant = determinant(minorMatrix)
            if (line + column) % 2 == 0:
                newMatrix[line][column] = minorDeterminant
            else:
                newMatrix[line][column] = -minorDeterminant

    return transposeMatrix(newMatrix)

### c) Function for multipling a matrix by a scalar

In [15]:
def multiplyMatrixByScalar (matrix, scalar):
    newMatrix = copy.deepcopy(matrix)
    for line in range(0, len(newMatrix)):
        for column in range(0, len(newMatrix)):
            newMatrix[line][column] = matrix[line][column] * scalar
    return newMatrix

#### Let's calculate $A^{-1}$ by using: $$ 
A^{-1} = \frac{1}{\det(A)} \times \operatorname{adj}(A) 
$$
 

In [16]:
adjugateMatrix = adjugate(matrixA)
aToMinus1 = multiplyMatrixByScalar(adjugateMatrix, 1 / matrixADetermiant)

#### Now all it remains is to calculate the results as the product $A^{-1} \cdot B$ and to extract the results

In [17]:
result = matrixVectorMultiplication(aToMinus1, matrixB)
xResult2 = result[0]
yResult2 = result[1]
zResult2 = result[2]
print(f'x = {xResult2}, y = {yResult2}, z = {zResult2}')

x = 0.35714285714285765, y = 2.071428571428571, z = 1.9285714285714293


##### We can see that the results are the same in both solutions

## Bonus
The formula given for the determinant is in fact just the calculation of a determinant using the decomposition by a line or column (by the first line in our case). More general the determant of a matrix is the sum of the minors of a line/column in the matrix.